In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
from keras.models import Sequential
from keras.layers import Dense,ReLU
from statistics import mode

In [5]:
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)
pd.options.mode.chained_assignment = None
np.random.seed(100)

In [6]:
mydata = pd.read_csv('heart.csv')
mydata

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [7]:
mydata.columns= ['age','sex','chest_pain_type','resting_blood_pressure','cholesterol','fasting_blood_sugar','rest_ecg','max_heart_rate','exercise_induced_angina','st_depression','st_slope','num_major_vessels','thalassemia','target']


In [8]:
mydata.columns

Index(['age', 'sex', 'chest_pain_type', 'resting_blood_pressure',
       'cholesterol', 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate',
       'exercise_induced_angina', 'st_depression', 'st_slope',
       'num_major_vessels', 'thalassemia', 'target'],
      dtype='object')

In [9]:
mydata.dtypes

age                          int64
sex                          int64
chest_pain_type              int64
resting_blood_pressure       int64
cholesterol                  int64
fasting_blood_sugar          int64
rest_ecg                     int64
max_heart_rate               int64
exercise_induced_angina      int64
st_depression              float64
st_slope                     int64
num_major_vessels            int64
thalassemia                  int64
target                       int64
dtype: object

In [10]:
mydata.describe()

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [11]:
correlation_matrix = mydata.corr()
correlation_target = correlation_matrix['target'].sort_values(ascending = False)
print(correlation_target)

target                     1.000000
chest_pain_type            0.433798
max_heart_rate             0.421741
st_slope                   0.345877
rest_ecg                   0.137230
fasting_blood_sugar       -0.028046
cholesterol               -0.085239
resting_blood_pressure    -0.144931
age                       -0.225439
sex                       -0.280937
thalassemia               -0.344029
num_major_vessels         -0.391724
st_depression             -0.430696
exercise_induced_angina   -0.436757
Name: target, dtype: float64


In [12]:
mydata['sex'][mydata['sex'] == 0] = 'female'
mydata['sex'][mydata['sex'] == 1] = 'male'
mydata['chest_pain_type'][mydata['chest_pain_type'] == 0] = 'typical angina'
mydata['chest_pain_type'][mydata['chest_pain_type'] == 1] = 'atypical angina'
mydata['chest_pain_type'][mydata['chest_pain_type'] == 2] = 'non-anginal pain'
mydata['chest_pain_type'][mydata['chest_pain_type'] == 3] = 'asymptomatic'
mydata['resting_blood_pressure'][mydata['resting_blood_pressure'] == 0] = 'lower than 120mg/ml'
mydata['resting_blood_pressure'][mydata['resting_blood_pressure'] == 1] = 'greater than 120mg/ml'
mydata['rest_ecg'][mydata['rest_ecg'] == 0] = 'normal'
mydata['rest_ecg'][mydata['rest_ecg'] == 1] = 'ST-T wave abnormality'
mydata['rest_ecg'][mydata['rest_ecg'] == 2] = 'left ventricular hypertrophy'
mydata['exercise_induced_angina'][mydata['exercise_induced_angina'] == 0] = 'no'
mydata['exercise_induced_angina'][mydata['exercise_induced_angina'] == 1] = 'yes'
mydata['st_slope'][mydata['st_slope'] == 0] = 'upsloping'
mydata['st_slope'][mydata['st_slope'] == 1] = 'flat'
mydata['st_slope'][mydata['st_slope'] == 2] = 'downsloping'
mydata['thalassemia'][mydata['thalassemia'] == 1] = 'normal'
mydata['thalassemia'][mydata['thalassemia'] == 2] = 'fixed defect'
mydata['thalassemia'][mydata['thalassemia'] == 1] = 'reversible defect'

In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [14]:
features = mydata[['sex','chest_pain_type','resting_blood_pressure','cholesterol','fasting_blood_sugar','rest_ecg','max_heart_rate','exercise_induced_angina','st_depression','st_slope','num_major_vessels','thalassemia']]
target = mydata['target']

In [15]:
features_encoded = pd.get_dummies(features,drop_first=True)

In [16]:
X_train, X_test ,y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

In [17]:
lin_model = LogisticRegression()
lin_model.fit(X_train,y_train)
print("Linear Model Accuracy: ",lin_model.score(X_test, y_test))

Linear Model Accuracy:  0.9016393442622951


/Users/fadel/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
svm_model = SVC()
svm_model.fit(X_train, y_train)
print("Support Vector Model Accuracy: ", svm_model.score(X_test, y_test))

Support Vector Model Accuracy:  0.6557377049180327


In [19]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
print("Naive Bayes Model Accuracy: ", nb_model.score(X_test, y_test))

Naive Bayes Model Accuracy:  0.8524590163934426


In [20]:
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)
print("Decision Tree Model Accuracy: ", tree_model.score(X_test, y_test))

Decision Tree Model Accuracy:  0.8360655737704918


In [21]:
forest_model = RandomForestClassifier()
forest_model.fit(X_train, y_train)
print("Random Forest Model Accuracy: ", forest_model.score(X_test, y_test))

Random Forest Model Accuracy:  0.8688524590163934
